## Context

This notebook is an end-to-end example that you can follow to create a project, upload a first plugin and activate it on this project, and later see the plugin in action and monitor it.

## Step 1 : Instantiate Kili with your API_KEY

In [ ]:
#%pip install kili

from kili.client import Kili
import os

api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'
# api_endpoint = 'https://cloud.kili-technology.com/api/label/v2/graphql'

api_key = input()

kili = Kili(api_endpoint=api_endpoint, api_key=api_key)

## Step 2 : Create the project

First, we need to create a new project. For our example, it will be a project of type `PDF` and it will have the following `jsonInterace` :

In [ ]:
json_interface = {
    "jobs": {
        "JOB_0": {
            "content": {
                "categories": {
                    "IBAN": {
                        "children": [
                            "TRANSCRIPTION_JOB"
                        ],
                        "name": "IBAN",
                        "color": "#3CD876",
                        "id": "category1"
                    },
                    "CURRENCY": {
                        "children": [
                            "TRANSCRIPTION_JOB_2"
                        ],
                        "name": "Currency",
                        "color": "#D33BCE",
                        "id": "category2"
                    },
                    "AMOUNT": {
                        "children": [
                            "TRANSCRIPTION_JOB_3"
                        ],
                        "name": "Amount",
                        "color": "#FB753C",
                        "id": "category3"
                    }
                },
                "input": "radio"
            },
            "instruction": "Payment information",
            "isChild": False,
            "tools": [
                "rectangle"
            ],
            "mlTask": "OBJECT_DETECTION",
            "models": {},
            "isVisible": True,
            "required": 1,
            "isNew": False
        },
        "TRANSCRIPTION_JOB": {
            "content": {
                "input": "textField"
            },
            "instruction": "",
            "mlTask": "TRANSCRIPTION",
            "required": 1,
            "isChild": True,
            "isNew": False
        },
        "OBJECT_DETECTION_JOB": {
            "content": {
                "categories": {
                    "BILLING_DATE": {
                        "children": [
                            "TRANSCRIPTION_JOB_0"
                        ],
                        "color": "#472CED",
                        "name": "Billing date",
                        "id": "category4"
                    },
                    "PAYMENT_DATE": {
                        "children": [
                            "TRANSCRIPTION_JOB_1"
                        ],
                        "name": "Payment date",
                        "color": "#5CE7B7",
                        "id": "category5"
                    }
                },
                "input": "radio"
            },
            "instruction": "Date information",
            "mlTask": "OBJECT_DETECTION",
            "required": 1,
            "tools": [
                "rectangle"
            ],
            "isChild": False,
            "isNew": False
        },
        "TRANSCRIPTION_JOB_0": {
            "content": {
                "input": "date"
            },
            "instruction": "",
            "mlTask": "TRANSCRIPTION",
            "required": 1,
            "isChild": True,
            "isNew": False
        },
        "TRANSCRIPTION_JOB_1": {
            "content": {
                "input": "date"
            },
            "instruction": "",
            "mlTask": "TRANSCRIPTION",
            "required": 1,
            "isChild": True,
            "isNew": False
        },
        "TRANSCRIPTION_JOB_2": {
            "content": {
                "input": "textField"
            },
            "instruction": "",
            "mlTask": "TRANSCRIPTION",
            "required": 1,
            "isChild": True,
            "isNew": False
        },
        "TRANSCRIPTION_JOB_3": {
            "content": {
                "input": "number"
            },
            "instruction": "",
            "mlTask": "TRANSCRIPTION",
            "required": 1,
            "isChild": True,
            "isNew": False
        }
    }
}

In [ ]:
# Create the project 

title = 'Plugins test PDF'
description = 'First project with a plugin'
input_type = 'PDF'

project = kili.create_project(title=title,
                                    description=description,
                                    input_type=input_type,
                                    json_interface=json_interface)
project_id = project["id"]

print(f'Created project {project_id}')

In [ ]:
# Upload an asset

content_array = ['./datasets/plugins/bill_model.pdf']
names_array = ['bill_model.pdf']

kili.append_many_to_dataset(
    project_id=project_id,
    content_array=content_array,
    external_id_array=names_array
)

asset_id = list(kili.assets(project_id=project_id, fields=["id"], disable_tqdm=True))[0]["id"]

# Add `accuracy` to the metadata type of the assets
kili.update_properties_in_project(
  project_id = project_id,
  metadata_types = {
    'accuracy': 'number',
  }
)

This project has two jobs with several tasks of transcription associated. We will be interested in the *Payment information* job, and more precisely the *IBAN* and the *Currency* sub-jobs. With our plugin, we want to be sure that the labelers write the correct values for the two transcriptions (so we are interested only in the `on_submit` handler of the plugin), since we know for sure that the IBAN should start with the 2 letters *FR*, and that the currency should be one of : *EUR*, *USD*. At then end, we also calculate an accuracy and we insert it in the `json_metadata` of the asset.

To iterate on the plugin code, you can refer to *plugins_development.ipynb* notebook

## The plugin
The final plugin is in the `./datasets/plugins/plugin.py` file and its content is the following :

```python
from kili.plugins import PluginCore
from kili.types import Label


def check_rules_on_label(label: Label):
    """
    my custom handle method
    """
    label_id = label["id"]
    jsonResponse = label["jsonResponse"]

    print(f"Started searching for issues for label {label_id}")

    annotations_for_job0 = jsonResponse["JOB_0"]["annotations"]

    issues_array = []
    mid_issues_array = []

    for bbox in annotations_for_job0:

        if bbox["categories"][0]["name"] == "IBAN":
            iban = bbox["children"]["TRANSCRIPTION_JOB"]["text"]

            if iban[0:2] != "FR":
                issues_array.append("IBAN number should start by FR")
                mid_issues_array.append(bbox["mid"])

        if bbox["categories"][0]["name"] == "CURRENCY":
            currency = bbox["children"]["TRANSCRIPTION_JOB_2"]["text"]

            if currency not in ["USD", "EUR"]:
                issues_array.append("Authorized currency are only Euro and Dollar")
                mid_issues_array.append(bbox["mid"])

    print(f"Finished searching for issues for label {label_id}")
    return issues_array, mid_issues_array


class PluginHandler(PluginCore):
    """
    Custom plugin instance
    """

    def on_review(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Review action
        """
        super().on_review(label, asset_id)
        self.logger.info("No action on review for now")

    def on_submit(self, label: Label, asset_id: str) -> None:
        """
        Dedicated handler for Submit action
        """
        self.logger.info("On submit called")

        label_id = label["id"]
        annotations_for_job0 = label["jsonResponse"]["JOB_0"]["annotations"]

        issues_array, mid_issues_array = check_rules_on_label(label)

        project_id = self.project_id

        if len(issues_array) > 0:
            print("Creating an issue...")

            for i, _ in enumerate(issues_array):

                self.kili.append_to_issues(
                    label_id=label_id,
                    project_id=project_id,
                    object_mid=mid_issues_array[i],
                    text=issues_array[i],
                )

            self.kili.send_back_to_queue(asset_ids=[asset_id])

            print("Issue created!")

        accuracy = 100 - len(issues_array) / len(annotations_for_job0) * 100

        self.kili.update_properties_in_assets(
            asset_ids=[asset_id], json_metadatas=[{"accuracy": accuracy}]
        )

```

## Step 3 : Uploading the plugin

With the plugin defined in a separate `python` file, we can now upload it and activate it on our project.

In [ ]:
path_to_plugin = './datasets/plugins/plugin.py'
plugin_name = 'Plugin_IBAN_Currency'

kili.upload_plugin_beta(path_to_plugin, plugin_name)

kili.activate_plugin_on_project(plugin_name, project_id=project_id)

## Step 4 : Seeing the plugin in action
You should wait for the plugin to be successfully deployed. After that, you can test it by labelling in the Kili interface or just by uploading the following label. You can see after that in the Kili app the issues created by the plugin (in case there are some errors).

In [ ]:
json_response = {
  "JOB_0": {
    "annotations": [
      {
        "annotations": [
          {
            "boundingPoly": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.6981818181818183,
                      "y": 0.6088564971238956
                    },
                    {
                      "x": 0.6981818181818183,
                      "y": 0.6263405519399533
                    },
                    {
                      "x": 0.9360000000000002,
                      "y": 0.6088564971238956
                    },
                    {
                      "x": 0.9360000000000002,
                      "y": 0.6263405519399533
                    }
                  ]
                ]
              }
            ],
            "pageNumberArray": [
              1
            ],
            "polys": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.6981818181818183,
                      "y": 0.6088564971238956
                    },
                    {
                      "x": 0.6981818181818183,
                      "y": 0.6263405519399533
                    },
                    {
                      "x": 0.9360000000000002,
                      "y": 0.6088564971238956
                    },
                    {
                      "x": 0.9360000000000002,
                      "y": 0.6263405519399533
                    }
                  ]
                ]
              }
            ]
          }
        ],
        "categories": [
          {
            "confidence": 100,
            "name": "IBAN"
          }
        ],
        "content": "",
        "mid": "20221109154444106-16050",
        "children": {
          "TRANSCRIPTION_JOB": {
            "text": "IBAN",
            "categories": []
          }
        }
      },
      {
        "annotations": [
          {
            "boundingPoly": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.9120000000000001,
                      "y": 0.5373775671406004
                    },
                    {
                      "x": 0.9120000000000001,
                      "y": 0.5533189112375942
                    },
                    {
                      "x": 0.9301818181818183,
                      "y": 0.5373775671406004
                    },
                    {
                      "x": 0.9301818181818183,
                      "y": 0.5533189112375942
                    }
                  ]
                ]
              }
            ],
            "pageNumberArray": [
              1
            ],
            "polys": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.9120000000000001,
                      "y": 0.5373775671406004
                    },
                    {
                      "x": 0.9120000000000001,
                      "y": 0.5533189112375942
                    },
                    {
                      "x": 0.9301818181818183,
                      "y": 0.5373775671406004
                    },
                    {
                      "x": 0.9301818181818183,
                      "y": 0.5533189112375942
                    }
                  ]
                ]
              }
            ]
          }
        ],
        "categories": [
          {
            "confidence": 100,
            "name": "CURRENCY"
          }
        ],
        "content": "",
        "mid": "20221109154457442-60957",
        "children": {
          "TRANSCRIPTION_JOB_2": {
            "text": "euro",
            "categories": []
          }
        }
      },
      {
        "annotations": [
          {
            "boundingPoly": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8443636363636365,
                      "y": 0.5348063826088272
                    },
                    {
                      "x": 0.8443636363636365,
                      "y": 0.5533189112375942
                    },
                    {
                      "x": 0.909818181818182,
                      "y": 0.5348063826088272
                    },
                    {
                      "x": 0.909818181818182,
                      "y": 0.5533189112375942
                    }
                  ]
                ]
              }
            ],
            "pageNumberArray": [
              1
            ],
            "polys": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8443636363636365,
                      "y": 0.5348063826088272
                    },
                    {
                      "x": 0.8443636363636365,
                      "y": 0.5533189112375942
                    },
                    {
                      "x": 0.909818181818182,
                      "y": 0.5348063826088272
                    },
                    {
                      "x": 0.909818181818182,
                      "y": 0.5533189112375942
                    }
                  ]
                ]
              }
            ]
          }
        ],
        "categories": [
          {
            "confidence": 100,
            "name": "AMOUNT"
          }
        ],
        "content": "",
        "mid": "20221109154503621-2805",
        "children": {
          "TRANSCRIPTION_JOB_3": {
            "text": "480",
            "categories": []
          }
        }
      }
    ]
  },
  "OBJECT_DETECTION_JOB": {
    "annotations": [
      {
        "annotations": [
          {
            "boundingPoly": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8254545454545456,
                      "y": 0.2329493185786526
                    },
                    {
                      "x": 0.8254545454545456,
                      "y": 0.24889066267564647
                    },
                    {
                      "x": 0.930909090909091,
                      "y": 0.2329493185786526
                    },
                    {
                      "x": 0.930909090909091,
                      "y": 0.24889066267564647
                    }
                  ]
                ]
              }
            ],
            "pageNumberArray": [
              1
            ],
            "polys": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8254545454545456,
                      "y": 0.2329493185786526
                    },
                    {
                      "x": 0.8254545454545456,
                      "y": 0.24889066267564647
                    },
                    {
                      "x": 0.930909090909091,
                      "y": 0.2329493185786526
                    },
                    {
                      "x": 0.930909090909091,
                      "y": 0.24889066267564647
                    }
                  ]
                ]
              }
            ]
          }
        ],
        "categories": [
          {
            "confidence": 100,
            "name": "BILLING_DATE"
          }
        ],
        "content": "",
        "mid": "20221109154513718-31859",
        "children": {
          "TRANSCRIPTION_JOB_0": {
            "text": "2021-07-12",
            "categories": []
          }
        }
      },
      {
        "annotations": [
          {
            "boundingPoly": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8247272727272729,
                      "y": 0.25094761030106505
                    },
                    {
                      "x": 0.8247272727272729,
                      "y": 0.26431776986628575
                    },
                    {
                      "x": 0.9287272727272728,
                      "y": 0.25094761030106505
                    },
                    {
                      "x": 0.9287272727272728,
                      "y": 0.26431776986628575
                    }
                  ]
                ]
              }
            ],
            "pageNumberArray": [
              1
            ],
            "polys": [
              {
                "normalizedVertices": [
                  [
                    {
                      "x": 0.8247272727272729,
                      "y": 0.25094761030106505
                    },
                    {
                      "x": 0.8247272727272729,
                      "y": 0.26431776986628575
                    },
                    {
                      "x": 0.9287272727272728,
                      "y": 0.25094761030106505
                    },
                    {
                      "x": 0.9287272727272728,
                      "y": 0.26431776986628575
                    }
                  ]
                ]
              }
            ]
          }
        ],
        "categories": [
          {
            "confidence": 100,
            "name": "PAYMENT_DATE"
          }
        ],
        "content": "",
        "mid": "20221109154526628-20969",
        "children": {
          "TRANSCRIPTION_JOB_1": {
            "text": "2021-07-26",
            "categories": []
          }
        }
      }
    ]
  }
}

In [ ]:
kili.append_labels(
   json_response_array=[json_response],
   asset_id_array=[asset_id],
   label_type='DEFAULT'
)

If everything behaves as expected, the base version of the plugin should : 
 - create an issue with 2 comments
 - send back the asset back to the labeling queue. 

In [ ]:
# Let's check : 
print(kili.assets(project_id=project_id, asset_id=asset_id, fields=['status', 'issues.comments.text']))

# You can also access the app at the link 
print(f'Go to my project {api_endpoint.split("/api")[0]}/label/projects/{project_id}/menu/queue')

## Step 5 : Monitoring the plugin
In order to monitor a certain plugin, you can get its logs by using the following command :

In [ ]:
import json
from datetime import date
from datetime import datetime
dt = date.today()  # You can change this date if needed, or omit it to set it at the plugin creation date
start_date = datetime.combine(dt, datetime.min.time())

logs = kili.get_plugin_logs(project_id=project_id, plugin_name=plugin_name, start_date=start_date)

logs_json = json.loads(logs)
print(json.dumps(logs_json, indent=4))

## Step 6 : Managing the plugin
You can also use the following methods to better manage your plugins

In [ ]:
# Get the list of all uploaded plugins in your organization
plugins = kili.list_plugins()

In [ ]:
# Get the status of a plugin
status = kili.get_plugin_status(plugin_name=plugin_name)

In [ ]:
# Update a plugin with new source code
updated_path = 'path/to/updated/file.py'
kili.update_plugin(plugin_name=plugin_name, file_path=updated_path)

In [ ]:
# Deactivate the plugin on a certain project (the plugin can still be active for other projects)
kili.deactivate_plugin_on_project(plugin_name=plugin_name, project_id=project_id)

In [ ]:
# Delete the plugin completely (deactivates automatically the plugin from all projects)
kili.delete_plugin(plugin_name=plugin_name)